It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [49]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   From_To       5 non-null      object 
 1   FlightNumber  3 non-null      float64
 2   RecentDelays  5 non-null      object 
 3   Airline       5 non-null      object 
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes


<h3>1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).</h3>

In [51]:
for i in range(0,df['FlightNumber'].size):
    if (pd.isnull(df.iloc[i,1])):
        df.iloc[i,1] = df.iloc[i-1,1]+10

In [52]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


<h3>2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame. </h3>
<h3>3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)</h3>
<h3>4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.</h3>

In [53]:
#splitting the column using Series.str.split() _____2
Destination = []
From = []
city = df['From_To'].str.split('_')
for i in range(0,df['From_To'].size):
    From.append(city[i][0])
    for j in range(1,2):
        Destination.append(city[i][j])

        
print(From,Destination)

['LoNDon', 'MAdrid', 'londON', 'Budapest', 'Brussels'] ['paris', 'miLAN', 'StockhOlm', 'PaRis', 'londOn']


In [54]:
#assigning corrrect column name to dataframe
df['From'] = From
df ['Destination'] = Destination
#droping From_to Column
df.drop(['From_To'],axis=1,inplace = True)
df

,FlightNumber,RecentDelays,Airline,From,Destination
0,10045.0,"[23, 47]",KLM(!),LoNDon,paris
1,10055.0,[],<Air France> (12),MAdrid,miLAN
2,10065.0,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,10075.0,[13],12. Air France,Budapest,PaRis
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,londOn


In [55]:
#standardizing the strings column
df['From'] = df['From'].str.capitalize()
df['Destination'] = df['Destination'].str.capitalize()
df

,FlightNumber,RecentDelays,Airline,From,Destination
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,10055.0,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075.0,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


<h3>5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays. </h3>

In [4]:
delays = pd.DataFrame(df['RecentDelays'].to_list(),columns=['delay_1','delay_2','delay_3'])
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [5]:
delays.fillna(delays.mean(),inplace=True)
delays

,delay_1,delay_2,delay_3
0,23.00,47.000000,87.0
1,31.75,40.666667,87.0
2,24.00,43.000000,87.0
3,13.00,40.666667,87.0
4,67.00,32.000000,87.0
